In [1]:
import numpy as np  # 导入NumPy库，用于科学计算

from qibo import Circuit, gates  # 从qibo库导入Circuit和gates模块，用于构建量子电路
from qibo.optimizers import optimize  # 从qibo库导入optimize函数，用于优化参数
from qibo.quantum_info import infidelity  # 从qibo库导入infidelity函数，用于计算保真度

# custom loss function, computes fidelity
def myloss(parameters, circuit, target):
    circuit.set_parameters(parameters)
    final_state = circuit().state()
    return infidelity(final_state, target)

nqubits = 6
dims = 2**nqubits
nlayers  = 2

# Create variational circuit
circuit = Circuit(nqubits)
for l in range(nlayers):
    circuit.add(gates.RY(qubit, theta=0.0) for qubit in range(nqubits))
    circuit.add(gates.CZ(qubit, qubit + 1) for qubit in range(0, nqubits - 1, 2))
    circuit.add(gates.RY(qubit, theta=0.0) for qubit in range(nqubits))
    circuit.add(gates.CZ(qubit, qubit + 1) for qubit in range(1, nqubits - 2, 2))
    circuit.add(gates.CZ(0, nqubits - 1))
circuit.add(gates.RY(qubit, theta=0.0) for qubit in range(nqubits))

# Optimize starting from a random guess for the variational parameters
x0 = np.random.uniform(0, 2 * np.pi, nqubits * (2 * nlayers + 1))
data = np.random.normal(0, 1, size=dims)

# perform optimization
best, params, extra = optimize(myloss, x0, args=(circuit, data), method='BFGS')

# set final solution to circuit instance
circuit.set_parameters(params)

[Qibo 0.2.21|INFO|2025-10-15 10:44:53]: Using qibojit (numba) backend on /CPU:0


In [2]:
best, params, extra

(np.float64(-58.93523277356938),
 array([ 1.60720151,  1.11451961,  1.95481552,  1.37355815,  4.97179543,
         1.82582195,  4.8856967 , -1.89045016,  2.65316019,  0.77834956,
         1.42359459,  1.69791468,  5.50358049,  3.51754841,  2.23687874,
         1.12157054, -0.72208495,  1.0829569 , -0.50663992, -2.90450326,
         0.67632003,  1.7318636 ,  0.83112217,  6.74747292,  1.82293859,
         1.81834231,  0.45690667,  0.1418156 ,  2.64439599,  1.75808395]),
   message: Optimization terminated successfully.
   success: True
    status: 0
       fun: -58.93523277356938
         x: [ 1.607e+00  1.115e+00 ...  2.644e+00  1.758e+00]
       nit: 56
       jac: [ 7.153e-06 -4.768e-06 ... -5.722e-06  0.000e+00]
  hess_inv: [[ 1.923e-01  1.059e-01 ...  4.512e-02  6.626e-02]
             [ 1.059e-01  1.615e-01 ...  6.026e-02  3.458e-02]
             ...
             [ 4.512e-02  6.026e-02 ...  8.639e-02  1.394e-03]
             [ 6.626e-02  3.458e-02 ...  1.394e-03  1.395e-01]]
      